In [4]:
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import corpus as corpus_class
from filters import std_filters
import categories
import numpy as np
from scipy import sparse

In [2]:
corpus = corpus_class.corpus( categories.categories() )
corpus.load("question_train.csv", "question_category_train.csv")
corpus.process(**std_filters(), corpus_size=-1)
corpus.save()

In [5]:
corpus = corpus_class.load_from_file()

In [11]:
#clf = MultinomialNB(alpha=0.01)
clf = LogisticRegression(penalty='l1', C=4, solver='liblinear')
scores = []
for coprus in corpus.cv_split(3):
    corpus.make_features()
    clf.fit(corpus.X_tr, corpus.y_tr)
    y_pred = clf.predict(corpus.X_te)
    scores += [ f1_score(corpus.y_te, y_pred, average="macro") ]
scores = np.array(scores)
print(scores, scores.mean())

[ 0.51191244  0.5299111   0.53441785] 0.525413794828
